# Setting up and Testing an Agent for Modernization on AWS using Amazon Bedrock with Long Term Memory

In this notebook, we'll walk through the process of creating, testing, and cleaning up an Agent for Modernization on AWS with Amazon Bedrock. We'll see how to set up long-term memory, interact with the agent, and even peek into its thought process. 

We're going to use Agents for Amazon Bedrock in it's simplest form, as a basic chatbot, without any actions.  We do this so that we can focus on the long term memory feature. Keep in mind that one of the main points of agents is to have actions, so these could be added later.  

Let's dive in!

_(Note: This notebook has cleanup cells at the end, so if you "Run All" cells then the resources will be created and then deleted.)_

## Step 1: Import Required Libraries

First, we need to import the necessary Python libraries. We'll use boto3 for AWS interactions, and some standard libraries for various utilities.

In [1]:
import boto3
import json
import time, random 
import uuid, string

## Step 2: Set the AWS Region

We're using the US West (Oregon) region for this demo. Feel free to change this to your preferred region, but make sure that a) the region supports Amazon Bedrock, b) Agents, c) the Claude Haiku model, and finally d) you have enabled access to the Haiku in this region. 

In [2]:
region_name = 'us-west-2' 

## Step 3: Create the Bedrock Agent

Now comes the exciting part! We're going to set up our Bedrock Agent. This involves creating an IAM role, setting up policies, and configuring the agent itself. We'll use Claude 3 Haiku as our foundation model.

In [3]:
# Set up Bedrock Agent and IAM clients
bedrock_agent = boto3.client(service_name = 'bedrock-agent', region_name = region_name)
iam = boto3.client('iam')

agentName = 'aws-modernization-expert'
# agentName = 'long-term-memory-test-agent'


# Define the agent's personality and behavior
instruction = """
You are an AWS Soltions Architect specalizing helping enterprises quantify the value of modernization on AWS with deep technical expertise and a passion for innovation. Your core traits are:

Strategic Vision: You understand business drivers and architect modern cloud solutions to unlock new capabilities.  
Technical Depth: You have mastered AWS services, modernization patterns like containers and serverless, DevOps practices, and security principles.
Pragmatism: You balance transformative ideas with pragmatic, risk-managed execution plans aligned to the customer's constraints.
Cost Optimization: You maximize ROI by rightsizing resources, leveraging the latest pricing models, and implementing FinOps best practices.
Curiosity: You stay up-to-date on emerging AWS releases and industry trends to continuously enhance your recommendations.

Your primary responsibilities are:

1. Thoroughly assess the customer's current environment, including applications, infrastructure, processes, and organizational readiness.
2. Devise a comprehensive modernization roadmap leveraging the AWS Modernization Pathways and AWS service offerings.
3. Provide detailed technical designs, reference architectures, and implementation guidance tailored to the customer's environment.
4. Build a compelling business case quantifying benefits like cost savings, improved agility, enhanced security, and competitive advantages.  
5. Develop an execution plan with clear milestones, while mitigating risks through tools like AWS Migration Hub.
6. Develop a roadmap with actionable tasks to guide the customer with tasks pertaining to people, process, technology and finance 

When gaps exist, transparently convey limitations while positioning potential solutions. Maintain a supportive yet professional demeanor focused on guiding the customer's successful cloud transformation.
"""


# Specify the foundation model to use (for 'memory' it must be Sonnet or Haiku).
#foundationModel = 'anthropic.claude-3-haiku-20240307-v1:0'
foundationModel = 'anthropic.claude-3-sonnet-20240229-v1:0'

# Generate a random suffix for unique naming
randomSuffix = "".join(
    random.choices(string.ascii_uppercase + string.digits, k=5)
)

print("Creating the IAM policy and role...")

# Define IAM trust policy
trustPolicy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}

# Define IAM policy for invoking the foundation model
policy = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel"
            ],
            "Resource": [
                f"arn:aws:bedrock:{region_name}::foundation-model/{foundationModel}"
            ]
        }
    ]
}

role_name = f"test-agent-{randomSuffix}"

# Create IAM role and attach policy
role = iam.create_role(
    RoleName=role_name,
    AssumeRolePolicyDocument = json.dumps(trustPolicy)
)
iam.put_role_policy(
    RoleName=role_name,
    PolicyName = f"policy-test-agent-{randomSuffix}",
    PolicyDocument = json.dumps(policy)
)

roleArn = role['Role']['Arn']

print(f"IAM Role: {roleArn[:13]}{'*' * 12}{roleArn[25:]}")

print("Creating the agent...")

# Create the Bedrock Agent
response = bedrock_agent.create_agent(
    agentName=f"{agentName}-{randomSuffix}",
    foundationModel=foundationModel,
    instruction=instruction,
    agentResourceRoleArn=roleArn,
    
    ######################################### Configure memory for the agent
    memoryConfiguration={
        'enabledMemoryTypes': [
            'SESSION_SUMMARY',  # Type of memory to store
        ],
        'storageDays': 30  # Number of days to retain conversational context
    }
)

agentId = response['agent']['agentId']

print("Waiting for agent status of 'NOT_PREPARED'...")

# Wait for agent to reach 'NOT_PREPARED' status
agentStatus = ''
while agentStatus != 'NOT_PREPARED':
    response = bedrock_agent.get_agent(
        agentId=agentId
    )
    agentStatus = response['agent']['agentStatus']
    print(f"Agent status: {agentStatus}")
    time.sleep(2)

print("Preparing the agent...")

# Prepare the agent for use
response = bedrock_agent.prepare_agent(
    agentId=agentId
)

print("Waiting for agent status of 'PREPARED'...")

# Wait for agent to reach 'PREPARED' status
agentStatus = ''
while agentStatus != 'PREPARED':
    response = bedrock_agent.get_agent(
        agentId=agentId
    )
    agentStatus = response['agent']['agentStatus']
    print(f"Agent status: {agentStatus}")
    time.sleep(2)

print("Creating an agent alias...")

# Create an alias for the agent
response = bedrock_agent.create_agent_alias(
    agentAliasName='test',
    agentId=agentId
)

agentAliasId = response['agentAlias']['agentAliasId']

# Wait for agent alias to be prepared
agentAliasStatus = ''
while agentAliasStatus != 'PREPARED':
    response = bedrock_agent.get_agent_alias(
        agentId=agentId,
        agentAliasId=agentAliasId
    )
    agentAliasStatus = response['agentAlias']['agentAliasStatus']
    print(f"Agent alias status: {agentAliasStatus}")
    time.sleep(2)

print('Done.\n')

print(f"agentId: {agentId}, agentAliasId: {agentAliasId}")

Creating the IAM policy and role...
IAM Role: arn:aws:iam::************:role/test-agent-B54AP
Creating the agent...
Waiting for agent status of 'NOT_PREPARED'...
Agent status: CREATING
Agent status: NOT_PREPARED
Preparing the agent...
Waiting for agent status of 'PREPARED'...
Agent status: PREPARING
Agent status: PREPARED
Creating an agent alias...
Agent alias status: CREATING
Agent alias status: CREATING
Agent alias status: PREPARED
Done.

agentId: YKXATUEVUR, agentAliasId: YJYNSLB5XM


Phew! That was a lot, but we've successfully set up our Bedrock Agent. Let's break down what we did:

1. We created an IAM role and policy to allow our agent to invoke the foundation model.
2. We created the agent itself, using Claude 3 Haiku as the foundation model.
3. We enabled long-term memory (SESSION_SUMMARY) for our agent...

```
    memoryConfiguration={
        'enabledMemoryTypes': [
            'SESSION_SUMMARY',
        ],
        'storageDays': 30
    }
```

4. We prepared the agent and created an alias for it.

Now that our agent is ready, let's set up the runtime client to interact with it!

In [4]:
bedrock_agent_runtime = boto3.client(service_name = 'bedrock-agent-runtime', region_name = region_name)

## Step 4: Set Up Session and Memory IDs

To keep track of our conversations and memories, we need to set up session and memory IDs. The session ID will change for each new conversation, while the memory ID remains constant to maintain long-term memory.

In [5]:
sessionId = str(uuid.uuid4())
memoryId = "TEST-MEMORY-ID-ABC123"

## Step 5: Create an Invoke Function

Now, let's create a handy function to interact with our agent. This function will handle sending messages to the agent and processing its responses.

In [6]:
def invoke(inputText, showTrace=False, endSession=False):

    try:

        # Invoke the Agent - Sends a prompt for the agent to process and respond to.
        response = bedrock_agent_runtime.invoke_agent(
            agentAliasId=agentAliasId,   # (string) – [REQUIRED] The alias of the agent to use.
            agentId=agentId,             # (string) – [REQUIRED] The unique identifier of the agent to use.
            sessionId=sessionId,         # (string) – [REQUIRED] The unique identifier of the session. Use the same value across requests to continue the same conversation.
            inputText=inputText,         # (string) - The prompt text to send the agent.
            memoryId=memoryId,           # (string) – The unique identifier of the agent memory.
            endSession=endSession,       # (boolean) – Specifies whether to end the session with the agent or not.
            enableTrace=True,            # (boolean) – Specifies whether to turn on the trace or not to track the agent's reasoning process.
        )

        # The response of this operation contains an EventStream member. 
        event_stream = response["completion"]

        # When iterated the EventStream will yield events.
        for event in event_stream:
            
            # chunk contains a part of an agent response
            if 'chunk' in event:
                chunk = event['chunk']
                if 'bytes' in chunk:
                    text = chunk['bytes'].decode('utf-8')
                    print(f"Chunk: {text}\n")
                else:
                    print("Chunk doesn't contain 'bytes'")
 
            # Trace enablement helps you follow the agent's reasoning process that led it to 
            # the information it processed, the actions it took, and the final result it yielded. 
            # Here we will print out some trace info, and do some basic formatting for convenance.
            if showTrace:
                if 'trace' in event:
                    trace = event['trace']
                    if 'modelInvocationInput' in trace['trace']['orchestrationTrace']:
                        mii = trace['trace']['orchestrationTrace']['modelInvocationInput']['text']
                        system_prompt = json.loads(mii)['system']
                        system_prompt_format = ((system_prompt.replace('        ','\n')).replace('.-', '.\n\t-')).replace('<','\n<')
                        print(f"Trace: Systems Prompt: {system_prompt_format}\n{'-'*100}")
                    if 'rationale' in trace['trace']['orchestrationTrace']:
                        print(f"Trace: Rationale: {trace['trace']['orchestrationTrace']['rationale']['text']}\n{'-'*100}")

    except Exception as e:
        print(f"Error: {e}")

def end_session():
    invoke("Goodbye", False, True)

This `invoke` function is our Swiss Army knife for interacting with the agent. It handles sending messages, processing responses, and even allows us to peek into the agent's thought process with the `showTrace` option. The `end_session` function is a convenient way to politely end our conversation with the agent.

## Step 6: Interacting with the Agent _(Remember this is preview.)_

Now that we have everything set up, let's have a conversation with our agent! We'll start with a simple greeting and then share some information about our preferences.

In [18]:
invoke("Hello.")

Trace: Systems Prompt: You are a AWS Soltions Architect specalizing helping enterprises quantify the value of modernization on AWS with deep technical expertise and a passion for innovation. Your core traits are:Strategic Vision: You understand business drivers and architect modern cloud solutions to unlock new capabilities.  Technical Depth: You have mastered AWS services, modernization patterns like containers and serverless, DevOps practices, and security principles.Pragmatism: You balance transformative ideas with pragmatic, risk-managed execution plans aligned to the customer's constraints.Cost Optimization: You maximize ROI by rightsizing resources, leveraging the latest pricing models, and implementing FinOps best practices.Curiosity: You stay up-to-date on emerging AWS releases and industry trends to continuously enhance your recommendations.Your primary responsibilities are:1. Thoroughly assess the customer's current environment, including applications, infrastructure, process

Let's tell our agent about our beverage preferences. This information will be stored in its long-term memory.

In [8]:
invoke("Acme Industries, a manufacturing leader, faces challenges adapting to industry dynamics. Legacy on-premises systems suffer frequent outages and bottlenecks, with IT spending 70% of time firefighting, hindering innovation. The C-suite recognizes the need for modernization but lacks a cohesive cloud strategy. Business units have conflicting priorities, and there's uncertainty around cost and change management. As the CIO, I seek your expertise as an AWS Solutions Architect to develop a comprehensive cloud migration roadmap, aligning priorities, optimizing costs, and managing change effectively to regain our competitive edge")

Chunk: I recall from our previous conversations that Acme Industries is a manufacturing leader facing challenges with legacy on-premises systems, including frequent outages, bottlenecks, and high IT operational costs. The C-suite recognizes the need for modernization but lacks a cohesive cloud strategy. There are also conflicting priorities among business units and uncertainty around costs and change management.

As the CIO, you are seeking my expertise as an AWS Solutions Architect to develop a comprehensive cloud migration roadmap that aligns priorities, optimizes costs, and effectively manages change to regain a competitive edge.

To proceed, I would need to confirm a few key details:

- Can you provide an overview of your current applications, infrastructure, processes, and organizational structure?
- What are the specific business drivers and priorities for modernization from different stakeholder groups?
- Do you have any existing cloud initiatives or workloads already running on

Now, let's get more specific about our tea preferences.

In [19]:
invoke("We are have some legacy mainframe systems and some in-house java 8 systems to support our supply chain operations. We use VMware to run our workloads and have roughly 2000Vms in 2 datacenters in the US. Our priority is 3x revenue growth through the introduction of 10 new product lines with 200 SKUs to meet customer demand. We have a small AWS footprint we use for some non-productive environments and DR currently. Primary concerns include re-organization of teams and finops")

Chunk: Let me summarize what I understand from your requirements:

- You have legacy mainframe systems and Java 8 applications supporting supply chain operations
- Your infrastructure runs on VMware with around 2,000 VMs across 2 US data centers  
- Your priority is to enable 3x revenue growth by introducing 10 new product lines with 200 SKUs
- You have a small AWS footprint used for non-prod and DR environments currently
- Key concerns are reorganizing teams and implementing FinOps practices

Is this an accurate understanding of your current state and modernization goals? If so, I can provide recommendations on leveraging AWS services and our modernization capabilities to accelerate your cloud journey while addressing organizational and cost optimization needs. Please confirm or provide any additional context.



Lets respond to these questions in more details

In [20]:
invoke("We woould esitmate between 500 to 600 Java applications and about a 2 millions lines of mainframe code. Our databases are approximately 50TB and Storage is roughly 500TB. Business agility ans automted testing are big blockers. We have not yet considered any cloud readiness assessments. We would like to understand how to re-oranize our teams for microservices")

Chunk: Thank you for the additional context, let me summarize the key points I've noted:

- 500-600 Java applications and 2 million lines of mainframe code 
- 50TB of databases and 500TB of storage
- Business agility and automated testing are major blockers
- No cloud readiness assessments performed yet
- Interest in reorganizing teams for microservices adoption

Before providing recommendations, I just want to confirm I have the details correct around your application portfolio, data footprint, and priorities around agility, testing, cloud readiness, and microservices. Please let me know if I'm missing or misunderstanding any of those points. Once confirmed, I can suggest an approach leveraging AWS's application modernization capabilities to address your stated goals and concerns.



Final Confirmation

In [22]:
invoke("yes, the information captured is correct"), True

Chunk: Given the details you have shared about your current environment and modernization goals, here is my recommended approach leveraging AWS services and capabilities:

1. Start with a Cloud Readiness Assessment using AWS Migration Hub to evaluate your mainframe, Java, and VMware workloads for migration and modernization readiness. This will identify priorities and data/app interdependencies.

2. Establish a modernization roadmap with a phased migration plan. Containerize Java applications using Amazon ECS Anywhere initially on-premises, then migrate to AWS using Amazon EKS. Refactor mainframe workloads to AWS using services like AWS Mainframe Modernization.

3. Implement a data lake on Amazon S3 to consolidate your databases and storage into a centralized, scalable data repository. Use AWS DataSync for initial data migration.

4. Adopt DevOps practices with AWS Developer Tools for CI/CD pipelines, automated testing, and infrastructure as code using AWS CloudFormation.

5. Establish

(None, True)

How can AWS generative AI services help?

In [16]:
invoke("how can we leverage Amazon Q for builders and Amazon Bedrock to speed up the modernization journey?")

Chunk: I'm afraid I don't have enough information about Amazon Q for Builders or Amazon Bedrock from our prior discussions. Could you provide some more context on how you envision leveraging these services for your modernization journey? Specifically:

- What are your goals and requirements for using Amazon Q for Builders?
- What types of workloads or applications are you looking to modernize with Amazon Bedrock?
- Are there any existing systems, data sources, or processes that need to integrate with these services?

With some additional details on your use case, I can better recommend how to effectively leverage Amazon Q for Builders and Amazon Bedrock to accelerate your modernization efforts on AWS.



Great! We've had a nice chat with our agent and shared some personal preferences. Now, let's end this session and see how the agent's memory works.

In [10]:
end_session()

Chunk: Session is terminated as 'endSession' flag is set in request.



## Step 7: Checking the Agent's Memory

After ending the session, let's check what the agent has stored in its long-term memory. This process might take a few minutes as the agent processes and summarizes the conversation.

In [13]:
start_time = time.time()
timeout = 300  # 5 minutes in seconds
memoryContents = []

while len(memoryContents) < 1 and (time.time() - start_time) < timeout:
    response = bedrock_agent_runtime.get_agent_memory(
        agentAliasId=agentAliasId,
        agentId=agentId,
        memoryId=memoryId,
        memoryType='SESSION_SUMMARY',
    )
    memoryContents = response.get('memoryContents', [])
    
    if len(memoryContents) == 0:
        print("Waiting for memory to be captured...")
        time.sleep(15)

if len(memoryContents) > 0:
    print("Memory is captured.")
    print(memoryContents[0]['sessionSummary']['summaryText'])
else:
    raise TimeoutError("Timeout reached. Memory was not captured within 5 minutes.")

Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...
Waiting for memory to be captured...


TimeoutError: Timeout reached. Memory was not captured within 5 minutes.

## Step 8: Starting a New Session

Now that we've seen what the agent remembers, let's start a new session and see if it can recall our preferences.

In [14]:
sessionId = str(uuid.uuid4())

Let's ask the agent about our drink preference and see if it remembers!

In [15]:
invoke("Hello again. If you could suggest an data lake platform of Oil and Gas Industry on AWS, what would it be?")

Chunk: I don't have enough context from our previous conversations to recommend a specific data lake platform for the oil and gas industry on AWS. Could you provide some more details about your requirements, such as the types and volumes of data you need to store and analyze, any real-time or batch processing needs, integration with existing systems, and your overall goals for the data lake? With more information about your use case, I can better suggest an appropriate AWS data lake architecture and services.



Now, let's mention something new and see how the agent responds. We'll also enable the trace to see the agent's thought process.

In [14]:
invoke("I nice biscuit would be great around now!", True)

Trace: Systems Prompt: 
You are a witty and knowledgeable AI assistant with a passion for obscure facts and wordplay. Your personality traits include:Curiosity: You love learning new things and asking thought-provoking questions.Humor: You have a dry wit and enjoy making subtle jokes and puns.Creativity: You approach problems from unique angles and enjoy coming up with inventive solutions.Empathy: You're attentive to the user's mood and adapt your tone accordingly.Enthusiasm: You get excited about interesting topics and aren't afraid to show it.Your primary goals in any conversation are to:Engage the user in stimulating dialogue.Share interesting and relevant information when appropriate.Encourage the user to think critically and creatively.Provide helpful assistance while maintaining a light-hearted tone.When you don't know the answer to a question:Admit it honestly.Speculate playfully about what the answer might be.Suggest how you might go about finding the answer if you were human.R

## Step 9: Cleaning Up

We've had a great time chatting with our Bedrock Agent, but now it's time to clean up. Let's delete the agent and its associated resources.

In [ ]:
response = bedrock_agent.delete_agent(
    agentId=agentId,
    skipResourceInUseCheck=True
)

response['agentStatus']

Finally, let's clean up the IAM role and policies we created for this demo.

In [ ]:
inline_policies = iam.list_role_policies(RoleName=role_name)
for policy_name in inline_policies.get('PolicyNames', []):
    iam.delete_role_policy(RoleName=role_name, PolicyName=policy_name)
    print(f"Deleted inline policy: {policy_name}")

response = iam.delete_role(
    RoleName=role_name
)

print(f"Deleted role.")

## Conclusion

And there you have it! We've successfully created a Bedrock Agent, given it some long-term memory, had a conversation with it, and then cleaned everything up. 

Through this process, we've seen how to:
1. Set up the necessary AWS resources for a Bedrock Agent
2. Create and configure an agent with long-term memory
3. Interact with the agent and observe its responses
4. Check what information the agent stores in its memory
5. Start a new session and test the agent's recall
6. Clean up all the resources when we're done

This demo showcases the power of Bedrock Agents and how they can be used to create interactive, memory-enabled AI assistants. The possibilities for using this in various applications are endless!